In [2]:
import pandas as pd
assetrelationship = pd.read_csv("C:/Users/jesse/Downloads/output_ml_students/output_ml_students/backend_assetrelationship.csv")
asset = pd.read_csv("C:/Users/jesse/Downloads/output_ml_students/output_ml_students/backend_asset.csv")
assettype = pd.read_csv("C:/Users/jesse/Downloads/output_ml_students/output_ml_students/backend_assettype.csv")
assetrelationshipservice = pd.read_csv("C:/Users/jesse/Downloads/output_ml_students/output_ml_students/backend_assetrelationshipservice.csv")
assetproperty = pd.read_csv("C:/Users/jesse/Downloads/output_ml_students/output_ml_students/backend_assetproperty.csv")
assetrelationshipservicemetric = pd.read_csv("C:/Users/jesse/Downloads/output_ml_students/output_ml_students/backend_assetrelationshipservicemetric.csv")


C:\Users\jesse\AppData\Local\Temp\ipykernel_17448\2321467457.py:3: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  asset = pd.read_csv("C:/Users/jesse/Downloads/output_ml_students/output_ml_students/backend_asset.csv")


In [3]:
# All assettype ids that are OT
OT_types = []
for i in range(len(assettype["id"])):
    if assettype["category"][i] == "OT":
        OT_types.append(assettype["id"][i])

In [5]:
# All OT assets in a list
OT_assets_list = []
for i in range(len(asset["asset_type_id"])):
    if asset["asset_type_id"][i] in OT_types:
        OT_assets_list.append(asset["id"][i])
len(OT_assets_list)

52

In [6]:
# All Ot assets in a dataframe
OT_assets = asset[asset["id"].isin(OT_assets_list)]

In [7]:
# The parents of all OT assets in a list and dataframe
OT_parents_list = OT_assets["parent_asset_id"].unique().tolist()[1:]
OT_parents = asset[asset["id"].isin(OT_parents_list)]
len(OT_parents)

21

In [264]:
# Add a column "asset_type" to the assets dataframe
asset_and_type = asset.merge(assettype[["id","category"]],left_on="asset_type_id",right_on="id",how="left")
asset_and_type = asset_and_type.drop(columns="id_y").rename(columns={"id_x":"id"})
asset_and_type["category"] = asset_and_type["category"].fillna("Unspecified")
# See how many of each category there is
all_categories = asset_and_type["category"].unique().tolist()
category_counts = {}
for i in all_categories:
    category_counts[i] = asset_and_type["category"].value_counts().get(i,0)
category_counts

{'Other': 192,
 'Network': 68015,
 'OT': 52,
 'IT': 270,
 'IoT': 33,
 'Unspecified': 50563}

In [270]:
vlan_ids

['{2102}',
 '{120}',
 '{2101}',
 '{45}',
 '{51}',
 '{2200}',
 '{49}',
 '{2103}',
 '{1001}',
 '{1007}',
 '{48}',
 nan,
 '{}',
 '{50}',
 '{2201}',
 '{1070}',
 '{70}']

In [283]:
# Seperate by a few vlan values
# vlan = 2102, 45, 51, 70
vlan_ids = asset_and_type["vlan_ids"].unique().tolist()
asset_vlan_1007 = asset_and_type[asset_and_type["vlan_ids"] == "{1007}"]
len(asset_vlan_1007)

9563

In [274]:
# Get a dataframe with just the asset_id and the category
nodes_table_1007 = asset_vlan_1007[["id","category"]]
len(nodes_table_1007)
nodes_table_1007["category"].unique().tolist()

['Network']

In [275]:
# Find all relationships between vlan = 2102 assets
vlan_1007_relationships = assetrelationship[assetrelationship["source_asset_id"].isin(asset_vlan_1007["id"]) & assetrelationship["target_asset_id"].isin(asset_vlan_1007["id"])]
# Rename to SOURCE and TARGET node for visualization
vlan_1007_relationships = vlan_1007_relationships.rename(columns={"source_asset_id":"SOURCE","target_asset_id":"TARGET"})

In [276]:
node_table_map_1007 = dict(zip(nodes_table_1007["id"],nodes_table_1007["category"]))
vlan_1007_relationships["SOURCE_type"] = vlan_1007_relationships["SOURCE"].map(node_table_map_1007)
vlan_1007_relationships["TARGET_type"] = vlan_1007_relationships["TARGET"].map(node_table_map_1007)

In [277]:
# Export to excel file
vlan_1007_relationships.to_excel("vlan_1007.xlsx",index=False)
nodes_table_1007.to_excel("nodes_table_1007.xlsx",index=False)